In [1]:
import sys
sys.path.append("../")  # 添加项目根目录到路径中

In [2]:
import torch
import torch.optim as optim
from torch import nn

import hgp
from hgp.models import HGNNP,CHGNN
from hgp.function import StraightThroughEstimator

import numpy as np
import random
DEVICE = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
DEVICE


/home/ycq/work/graph-partition-with-gcn/.env-HGP/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cuda', index=0)

In [3]:
seed = 42
torch.manual_seed(seed) # 为CPU设置随机种子
torch.cuda.manual_seed(seed) # 为当前GPU设置随机种子
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU，为所有GPU设置随机种子
np.random.seed(seed)  # Numpy module.
random.seed(seed)  # Python random module.	
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [4]:
from hgp.models import ParameterDict

# fmt: off
h_hyper_prmts = ParameterDict()
l_hyper_prmts = ParameterDict()

weight = 65
"""
h_hyper_prmts["convlayers11"] = {"in_channels": 2048, "out_channels": 2048, "use_bn": False, "drop_rate": 0.2}
# h_hyper_prmts["convlayers14"] = {"in_channels": 1024, "out_channels": 512, "use_bn": False, "drop_rate": 0.05}
h_hyper_prmts["convlayers1"] = {"in_channels": 2048, "out_channels": 2048, "use_bn": False, "drop_rate": 0.05}

l_hyper_prmts["linerlayer113"] = {"in_channels":2048, "out_channels":2048, "use_bn":True, "drop_rate":0.05}
l_hyper_prmts["linerlayer13"] = {"in_channels":2048, "out_channels":1024, "use_bn":True, "drop_rate":0.05}
l_hyper_prmts["linerlayer1"] = {"in_channels":1024, "out_channels":512, "use_bn":True, "drop_rate":0.05}
l_hyper_prmts["linerlayer12334"] = {"in_channels":512, "out_channels":512, "use_bn":True, "drop_rate":0.05}
l_hyper_prmts["linerlayer12"] = {"in_channels":512, "out_channels":256, "use_bn":True, "drop_rate":0.05}
l_hyper_prmts["linerlayer123"] = {"in_channels":256, "out_channels":128, "use_bn":True, "drop_rate":0.05}
l_hyper_prmts["linerlayer121"] = {"in_channels":128, "out_channels":64, "use_bn":False, "drop_rate":0.05}
l_hyper_prmts["linerlayer31"] = {"in_channels":64, "out_channels":2, "use_bn":False, "drop_rate":0.05}
"""

partitions = 6

h_hyper_prmts["convlayers1"] = {"in_channels": 1019, "out_channels": 512, "use_bn": False, "drop_rate": 0.2}
h_hyper_prmts["convlayers12"] = {"in_channels": 512, "out_channels": 256, "use_bn": False, "drop_rate": 0.1}
h_hyper_prmts["convlayers13"] = {"in_channels": 256, "out_channels": 256, "use_bn": False, "drop_rate": 0.1}
h_hyper_prmts["convlayers2"] = {"in_channels": 256, "out_channels": 256, "use_bn": False, "drop_rate": 0.1}
h_hyper_prmts["convlayers3"] = {"in_channels": 256, "out_channels": 128, "use_bn": False, "drop_rate": 0.1}
h_hyper_prmts["convlayers5"] = {"in_channels": 128, "out_channels": 64, "use_bn": False, "drop_rate": 0.1}
#h_hyper_prmts["convlayers51"] = {"in_channels": 64, "out_channels": 32, "use_bn": False, "drop_rate": 0.1}
#h_hyper_prmts["convlayers52"] = {"in_channels": 64, "out_channels": 64, "use_bn": False, "drop_rate": 0.1}
#h_hyper_prmts["convlayers53"] = {"in_channels": 32, "out_channels": 64, "use_bn": False, "drop_rate": 0.1}
#h_hyper_prmts["convlayers54"] = {"in_channels": 256, "out_channels": 512, "use_bn": False, "drop_rate": 0.1}
#h_hyper_prmts["convlayers55"] = {"in_channels": 512, "out_channels": 64, "use_bn": False, "drop_rate": 0.1}


l_hyper_prmts["linerlayer1"] = {"in_channels":list(h_hyper_prmts.values())[-1]["out_channels"], "out_channels":64, "use_bn":True, "drop_rate":0.05}
#l_hyper_prmts["linerlayer12"] = {"in_channels":32, "out_channels":64, "use_bn":False, "drop_rate":0.05}
#l_hyper_prmts["linerlayer2"] = {"in_channels":64, "out_channels":128, "use_bn":False, "drop_rate":0.05}
#l_hyper_prmts["linerlayer21"] = {"in_channels":128, "out_channels":256, "use_bn":False, "drop_rate":0.05}
#l_hyper_prmts["linerlayer22"] = {"in_channels":256, "out_channels":512, "use_bn":False, "drop_rate":0.05}
#l_hyper_prmts["linerlayer23"] = {"in_channels":512, "out_channels":256, "use_bn":False, "drop_rate":0.05}
l_hyper_prmts["linerlayer24"] = {"in_channels":64, "out_channels":64, "use_bn":False, "drop_rate":0.05}
l_hyper_prmts["linerlayer33"] = {"in_channels":64, "out_channels":32, "use_bn":False, "drop_rate":0.05}
l_hyper_prmts["linerlayer4"] = {"in_channels":32, "out_channels":6, "use_bn":True, "drop_rate":0.05}


hyper = {
    "h_hyper_prmts": h_hyper_prmts,
    "l_hyper_prmts":l_hyper_prmts,
    "init_features_dim":list(h_hyper_prmts.values())[0]["in_channels"],
    "partitions":partitions
}

# fmt: on

In [5]:
def loss_bs_matrix(outs, hg, device,weight):
    # fmt: off
    r"""
    对于超图的损失函数的矩阵形式.
    
    Args:
        ``outs``(`torch.nn.Module`):  模型的输出. Size :math:`(N, nums_classes)`.   
        ``hg``(`Hypergraph`):  超图对象.  
    """
    # fmt: on
    H = hg.H.to_dense().to(device)
    outs = outs.to(device)
    nn = torch.matmul(outs, (1 - torch.transpose(outs, 0, 1)))
    ne_k = torch.matmul(nn, H)
    ne_k = ne_k.mul(H)

    H_degree = torch.sum(H, dim=0)
    H_degree = H_degree

    H_1 = ne_k / H_degree
    a2 = 1 - H_1
    a3 = torch.prod(a2, dim=0)
    a3 = a3.sum()
    loss_1 = -1 * a3

    # pun = torch.mul(ne_k, H)

    # loss_1 = pun.sum()
    loss_2 = torch.var(torch.sum(outs, dim=0)).to(device)
    loss = weight * loss_1 + loss_2
    return loss, loss_1, loss_2

#### 定义用于训练的类Trainer

In [6]:
class Trainer(nn.Module):
    # fmt: off
    r"""
    用于承担训练的类.
    ---
    Args:
        ``net``: (``torch.nn.Module``): 网络模型.  
        ``X``: (``torch.Tensor``): 作为输入的顶点特征矩阵. Size :math:`(N, C_{in})`.  
        ``hg``: (``dhg.Hypergraph``): 包含 :math:`N` 个顶点的超图结构.  
    """
    # fmt: on
    def __init__(self, net, X, hg, optimizer):
        super().__init__()
        self.X: torch.Tensor = X.to(DEVICE)
        self.hg = hg.to(DEVICE)
        self.de = self.hg.H.to_dense().sum(dim=0).to("cpu").to(DEVICE)
        self.optimizer: torch.optim.Optimizer = optimizer
        self.layers = nn.ModuleList()
        self.layers.append(net.to(DEVICE))
        self.weight = 200
    def forward(self, X):
        X = self.layers[0](X, self.hg)
        for layer in self.layers[1:]:
            X = layer(X)
        return X

    def run(self, epoch):
        self.train()  # train mode | 设置为训练模式
        self.optimizer.zero_grad()
        outs = self.forward(self.X)
        loss, loss_1, loss_2 = loss_bs_matrix(outs, self.hg, device=DEVICE,weight=self.weight)
        loss.backward()
        self.optimizer.step()

        return loss.item(), loss_1.item(), loss_2.item()

#### 准备数据

In [7]:
import hgp.utils
G = hgp.utils.from_pickle_to_hypergraph("../data/citeseer")
edges, _ = G.e
G.num_e,G.num_v

(767, 1019)

In [8]:
X = torch.randn(size=(G.num_v, hyper["init_features_dim"]))
# X = torch.eye(hyper["init_features_dim"])
net = HGNNP(hyper["h_hyper_prmts"]).to(DEVICE)
hgnn_trainer = Trainer(net=net, X=X, hg=G, optimizer=None).to(DEVICE)
for (k,v) in hyper["l_hyper_prmts"].items():
    hgnn_trainer.layers.append(nn.BatchNorm1d(num_features=v["in_channels"]).to(DEVICE)) if v["use_bn"] else None
    hgnn_trainer.layers.append(nn.ReLU().to(DEVICE))
    if v["drop_rate"] > 0:
        hgnn_trainer.layers.append(nn.Dropout(v["drop_rate"]))
    hgnn_trainer.layers.append(nn.Linear(in_features=v["in_channels"],out_features=v["out_channels"],device=DEVICE))
hgnn_trainer.layers.append(nn.Softmax(dim=1))

ModuleList(
  (0): HGNNP(
    (layers): ModuleList(
      (0): HGNNPConv(
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.2, inplace=False)
        (theta): Linear(in_features=1019, out_features=512, bias=True)
      )
      (1): HGNNPConv(
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.1, inplace=False)
        (theta): Linear(in_features=512, out_features=256, bias=True)
      )
      (2): HGNNPConv(
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.1, inplace=False)
        (theta): Linear(in_features=256, out_features=256, bias=True)
      )
      (3): HGNNPConv(
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.1, inplace=False)
        (theta): Linear(in_features=256, out_features=256, bias=True)
      )
      (4): HGNNPConv(
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.1, inplace=False)
        (theta): Linear(in_features=256, out_features=128, bias=True)
      )
      (5): HGNNPConv(
        (act): ReLU(inpla

In [9]:
# hgnn_trainer.layers
# for n,p in hgnn_trainer.named_parameters():
#     print(n,p)
hgnn_trainer.weight = weight

In [10]:
temp_loss_total,temp_loss1,temp_loss2 = torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False)
optim1 = optim.Adam(hgnn_trainer.parameters(), lr=3e-4, weight_decay=5e-8)
hgnn_trainer.optimizer = optim1
for epoch in range(20000):
    if hgnn_trainer.weight > 0.0036:
        hgnn_trainer.weight = hgnn_trainer.weight - 0.001
    loss,loss_1,loss_2 = hgnn_trainer.run(epoch=epoch)
    temp_loss_total += loss
    temp_loss1 += loss_1
    temp_loss2 += loss_2
    if epoch % 10 == 0:
        print(f"in {epoch} epoch, average loss: {temp_loss_total.item() / 10}")
        print(f"                , loss1: {temp_loss1.item() / 10}")
        print(f"                , loss2: {temp_loss2.item() / 10}")
        print(f"                , weight: {hgnn_trainer.weight}")
        print(f"=================================")
        sys.stdout.flush()
        temp_loss_total,temp_loss1,temp_loss2 = torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False)

in 0 epoch, average loss: -28.438430786132812
                , loss1: -1.2083264350891114
                , loss2: 50.10157775878906
                , weight: 64.999
in 10 epoch, average loss: -563.48310546875
                , loss1: -12.681715393066407
                , loss2: 260.7440185546875
                , weight: 64.98899999999995
in 20 epoch, average loss: -1590.22890625
                , loss1: -25.072515869140624
                , loss2: 39.053759765625
                , weight: 64.9789999999999
in 30 epoch, average loss: -2837.98125
                , loss1: -43.99250793457031
                , loss2: 20.355488586425782
                , weight: 64.96899999999985
in 40 epoch, average loss: -3621.750390625
                , loss1: -55.974786376953126
                , loss2: 14.559417724609375
                , weight: 64.9589999999998
in 50 epoch, average loss: -4455.207421875
                , loss1: -69.30055541992188
                , loss2: 46.09632263183594
          

KeyboardInterrupt: 

In [ ]:
hgnn_trainer.eval()
outs = hgnn_trainer.forward(hgnn_trainer.X)
outs_straight = StraightThroughEstimator.apply(outs)
G_clone = G.clone()
edges, _  = G_clone.e
cut = 0
for vertices in edges:
    if torch.prod(outs_straight[list(vertices)], dim=0).sum() == 0:
        cut += 1
    else:
        G_clone.remove_hyperedges(vertices)
assert cut == G_clone.num_e
cut

83

In [ ]:
num_nodes = outs_straight.sum(dim=0)
print(num_nodes)
(torch.max(num_nodes).item() - torch.min(num_nodes).item()) / num_nodes.sum().item()


tensor([163., 182., 155., 164., 171., 184.], device='cuda:0',
       grad_fn=<SumBackward1>)


0.02845927379784102

In [ ]:
torch.var(torch.tensor([53.,56,53,54,56,55]))

tensor(1.9000)